#Importando Bibliotecas

In [135]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display

#Base de Dados

In [136]:
URL_BASES = "https://raw.githubusercontent.com/Vit4s/Uti-eicu/main/eicu-collaborative-research-database-demo-2.0.1/"
patient_csv = "patient.csv"
apacheApsVar = "apacheApsVar.csv"
infusiondrug = "infusiondrug.csv"
physicalExam = "physicalExam.csv"
lab = "lab.csv"
hospital = "hospital.csv"
diagnosis = "diagnosis.csv"


url_patient = URL_BASES + patient_csv
url_apacheApsVar = URL_BASES + apacheApsVar
url_infusiondrug = URL_BASES + infusiondrug
url_physicalExam = URL_BASES + physicalExam
url_lab = URL_BASES + lab
url_hospital = URL_BASES + hospital
url_diagnosis = URL_BASES + diagnosis



df_patient = pd.read_csv(url_patient)
df_apacheApsVar = pd.read_csv(url_apacheApsVar)
df_infusiondrug = pd.read_csv(url_infusiondrug)
df_physicalExam = pd.read_csv(url_physicalExam)
df_lab = pd.read_csv(url_lab)
df_hospital = pd.read_csv(url_hospital)
df_diagnosis = pd.read_csv(url_diagnosis)

# Characteristic

In [137]:
df_patient.columns

Index(['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age',
       'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx',
       'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset',
       'hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid'],
      dtype='object')

In [138]:
characteristic = df_patient [['patientunitstayid', 'gender', 'age', 'ethnicity']].copy()
#Substitua valores específicos por NaN
characteristic = characteristic.replace(-1, pd.NA)
characteristic

,patientunitstayid,gender,age,ethnicity
0,141764,Female,87,Caucasian
1,141765,Female,87,Caucasian
2,143870,Male,76,Caucasian
3,144815,Female,34,Caucasian
4,145427,Male,61,Caucasian
...,...,...,...,...
2515,3351763,Female,62,Caucasian
2516,3352230,Male,41,African American
2517,3352231,Male,41,African American
2518,3352333,Male,72,Caucasian


#Organ Dysfunction

Nervous System

In [139]:
glasgow = df_apacheApsVar[['patientunitstayid', 'eyes', 'motor', 'verbal']].copy()
glasgow ['Total'] = glasgow ['eyes'] + glasgow ['motor'] + glasgow ['verbal']

sofa = df_patient[['patientunitstayid']].copy()


motor = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Motor Score', case=False, na=False)]
motor = motor.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
motor = motor[['patientunitstayid', 'physicalexamtext']].copy()
motor = motor.rename(columns={'physicalexamtext': 'motor'})
motor = motor.groupby('patientunitstayid')['motor'].max().reset_index()
motor['motor'] = motor['motor'].astype(int)


eyes = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
eyes = eyes.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
eyes = eyes[['patientunitstayid', 'physicalexamtext']].copy()
eyes = eyes.rename(columns={'physicalexamtext': 'eyes'})
eyes = eyes.groupby('patientunitstayid')['eyes'].max().reset_index()
eyes['eyes'] = eyes['eyes'].astype(int)

verbal = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
verbal = verbal.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
verbal = verbal[['patientunitstayid', 'physicalexamtext']].copy()
verbal = verbal.rename(columns={'physicalexamtext': 'verbal'})
verbal = verbal.groupby('patientunitstayid')['verbal'].max().reset_index()
verbal['verbal'] = verbal['verbal'].astype(int)

gcs = pd.merge(motor, eyes, on='patientunitstayid', how='outer')
gcs = pd.merge(gcs, verbal, on='patientunitstayid', how='outer')
gcs ['Total'] = gcs ['motor'] + gcs ['eyes'] + gcs ['verbal']

def score(row):
  if((row['Total']<6)):
    return 4
  elif((row['Total']>=6) and (row['Total']<=9)):
    return 3
  elif((row['Total']>=10) and (row['Total']<=12)):
    return 2
  elif((row['Total']>=13) and (row['Total']<=14)):
    return 1
  else:
    return 0

sofa['Nervous'] = gcs.apply(score, axis=1)
sofa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientunitstayid  2520 non-null   int64  
 1   Nervous            1971 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 39.5 KB


Cardiovascular System

In [140]:
dopamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dopamine', case=False, na=False)]
dobutamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dobutamine', case=False, na=False)]
epinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Epinephrine', case=False, na=False)]
norepinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Norepinephrine', case=False, na=False)]

dopamine = dopamine.query('infusionoffset >= 0 and infusionoffset <= 1440')
dobutamine = dobutamine.query('infusionoffset >= 0 and infusionoffset <= 1440')
epinephrine = epinephrine.query('infusionoffset >= 0 and infusionoffset <= 1440')
norepinephrine = norepinephrine.query('infusionoffset >= 0 and infusionoffset <= 1440')

dopamine = dopamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
dobutamine = dobutamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
epinephrine = epinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
norepinephrine = norepinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()

dopamine['concentration.dop'] = dopamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
epinephrine['concentration.epi'] = epinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
dobutamine['concentration.dob'] = dobutamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
norepinephrine['concentration.nor'] = norepinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)


dopamine.rename(columns={'drugamount' : 'dopamine'}, inplace=True)
dobutamine.rename(columns={'drugamount' : 'dobutamine'}, inplace=True)
epinephrine.rename(columns={'drugamount' : 'epinephrine'}, inplace=True)
norepinephrine.rename(columns={'drugamount' : 'norepinephrine'}, inplace=True)

dopamine = dopamine.groupby('patientunitstayid')['concentration.dop'].max().reset_index()
dobutamine = dobutamine.groupby('patientunitstayid')['concentration.dob'].max().reset_index()
epinephrine = epinephrine.groupby('patientunitstayid')['concentration.epi'].max().reset_index()
norepinephrine = norepinephrine.groupby('patientunitstayid')['concentration.nor'].max().reset_index()

cardio = pd.merge(dopamine, dobutamine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, epinephrine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, norepinephrine, on='patientunitstayid', how='outer')

In [141]:
systolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('systolic', case=False, na=False)]
diastolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('diastolic', case=False, na=False)]

systolic = systolic.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
diastolic = diastolic.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')

systolic = systolic[['patientunitstayid', 'physicalexamtext']].copy()
diastolic = diastolic[['patientunitstayid', 'physicalexamtext']].copy()

systolic.rename(columns={'physicalexamtext' : 'systolic'}, inplace=True)
diastolic.rename(columns={'physicalexamtext' : 'diastolic'}, inplace=True)

systolic = systolic.groupby('patientunitstayid')['systolic'].min().reset_index()
diastolic = diastolic.groupby('patientunitstayid')['diastolic'].min().reset_index()

MAP = pd.merge(systolic, diastolic, on='patientunitstayid')

try:
  MAP["diastolic"] = MAP["diastolic"].astype(int)
except ValueError:
  MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')

try:
  MAP["systolic"] = MAP["systolic"].astype(int)
except ValueError:
  MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

#PAM = (Pressão Sistólica + 2 x Pressão Diastólica) / 3
cardio['MAP'] = (MAP['systolic'] + 2*(MAP['diastolic'])) / 3

In [142]:
def score(row):
  if((row['concentration.dop']>15) or (row['concentration.epi']>0.1) or (row['concentration.nor']>0.1)):
    return 4
  elif((row['concentration.dop']>5) or (row['concentration.epi']<=0.1) or (row['concentration.nor']<=0.1)):
    return 3
  elif((row['concentration.dop']<=5) or (row['concentration.dop']>0)):
    return 2
  elif(row['MAP']<70):
    return 1
  else:
    return 0

sofa['Cardiovascular'] = cardio.apply(score, axis=1)
cardio

,patientunitstayid,concentration.dop,concentration.dob,concentration.epi,concentration.nor,MAP
0,258915,1.6,NaN,0.016,0.016,65.333333
1,441732,1.6,NaN,0.016,NaN,90.333333
2,564216,NaN,NaN,NaN,NaN,78.333333
3,603255,NaN,NaN,NaN,NaN,85.666667
4,827085,NaN,NaN,NaN,NaN,70.000000
...,...,...,...,...,...,...
193,3233306,NaN,NaN,0.016,0.016,77.666667
194,3233997,NaN,NaN,0.016,0.016,118.666667
195,3235765,NaN,NaN,1.000,1.000,66.000000
196,3239262,NaN,NaN,1.000,1.000,87.333333


Respiratory System

In [143]:
pao2 = df_lab[df_lab['labname'].str.contains('^paO2$', case=False, na=False, regex=True)]
fio2 = df_lab[df_lab['labname'].str.contains('^FiO2$', case=False, na=False, regex=True)]

pao2 = pao2.query('labresultoffset >= 0 and labresultoffset <= 1440')
fio2 = fio2.query('labresultoffset >= 0 and labresultoffset <= 1440')

pao2 = pao2[['patientunitstayid', 'labresult']].copy()
fio2 = fio2[['patientunitstayid', 'labresult']].copy()

pao2.rename(columns={'labresult' : 'pao2'}, inplace=True)
fio2.rename(columns={'labresult' : 'fio2'}, inplace=True)

pao2 = pao2.groupby('patientunitstayid')['pao2'].min().reset_index()
fio2 = fio2.groupby('patientunitstayid')['fio2'].max().reset_index()

resp = pd.merge(pao2, fio2, on='patientunitstayid', how='outer')
resp['Total'] = resp['pao2']/resp['fio2']

def score(row):
    if row['Total'] < 100:
        return 4
    elif 100 <= row['Total'] < 200:
        return 3
    elif 200 <= row['Total'] < 300:
        return 2
    elif 300 <= row['Total'] < 400:
        return 1
    else:
        return 0

sofa['Respiratory'] = resp.apply(score, axis=1)

Coagulation

In [144]:
platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]
platelets = platelets.query('labresultoffset >= 0 and labresultoffset <= 1440')
platelets = platelets[['patientunitstayid', 'labresult']].copy()
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

def score(row):
    if row['platelets'] < 20:
        return 4
    elif 20 <= row['platelets'] < 50:
        return 3
    elif 50 <= row['platelets'] < 100:
        return 2
    elif 100 <= row['platelets'] < 150:
        return 1
    else:
        return 0

sofa['Coagulation'] = platelets.apply(score, axis=1)

Liver

In [145]:
bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]
bilirubin = bilirubin.query('labresultoffset >= 0 and labresultoffset <= 1440')
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

def score(row):
    if row['bilirubin'] >= 12.0:
        return 4
    elif 12.0 > row['bilirubin'] >= 6.0:
        return 3
    elif 6.0 > row['bilirubin'] >= 2.0:
        return 2
    elif 2.0 > row['bilirubin'] > 1.2:
        return 1
    else:
        return 0

sofa['Liver'] = bilirubin.apply(score, axis=1)

Renal Function

In [146]:
creatinine = df_lab[df_lab['labname'].str.contains('creatinine', case=False, na=False)]
creatinine = creatinine.query('labresultoffset >=0 and labresultoffset <= 1440')
creatinine = creatinine[['patientunitstayid','labresult']].copy()
creatinine.rename(columns={'labresult' : 'creatinine'}, inplace=True)
creatinine = creatinine.groupby('patientunitstayid')['creatinine'].max().reset_index()

def score(row):
  if row['creatinine'] >= 5.0:
    return 4
  elif 5.0 > row['creatinine'] >= 3.5:
    return 3
  elif 3.0 > row['creatinine'] >= 2.0:
    return 2
  elif 2.0 > row['creatinine'] > 1.2:
    return 1
  else:
    return 0

sofa['Renal'] = creatinine.apply(score, axis=1)

Score Final

In [147]:
sofa['sofascore'] = sofa['Nervous'] + sofa['Cardiovascular'] + sofa['Respiratory'] + sofa['Coagulation'] + sofa['Liver'] + sofa['Renal']
sofa = sofa[['patientunitstayid', 'sofascore']].copy()
sofa

,patientunitstayid,sofascore
0,141764,8.0
1,141765,8.0
2,143870,5.0
3,144815,5.0
4,145427,1.0
...,...,...
2515,3351763,NaN
2516,3352230,NaN
2517,3352231,NaN
2518,3352333,NaN


#Inflammation


White blood cells count

In [148]:
WBC_view = df_lab[df_lab['labname'].str.contains('WBC x 1000', case=False, na=False)].copy()
WBCurine_view = df_lab[df_lab['labname'].str.contains("WBC's in urine", case=False, na=False)].copy()
CRP_view = df_lab[df_lab['labname'].str.contains('CRP', case=False, na=False)].copy()
lactate_view = df_lab[df_lab['labname'].str.contains('lactate', case=False, na=False)].copy()
Temperature_view = df_lab[df_lab['labname'].str.contains('Temp', case=False, na=False)].copy()
ESR_view = df_lab[df_lab['labname'].str.contains('ESR', case=False, na=False)].copy()
bands_view = df_lab[df_lab['labname'].str.contains('-bands', case=False, na=False)].copy()



WBC_view = WBC_view[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'labresulttext', 'labmeasurenamesystem']]
WBC_view = WBC_view.rename(columns={'labresultoffset': 'WBCmins', 'labresult': 'WBC', 'labresulttext' : 'WBCresulttext', 'labmeasurenamesystem' : 'WBC.unidMedida'})

filtered_df = WBC_view[(WBC_view['WBCmins'] > 0) & (WBC_view['WBCmins'] < 1441)]

WBC_view = filtered_df.groupby('patientunitstayid')['WBC'].max().reset_index()


WBC_view

,patientunitstayid,WBC
0,141765,10.2
1,143870,11.7
2,144815,7.9
3,145427,21.1
4,147306,13.5
...,...,...
1951,3351297,7.5
1952,3351763,8.5
1953,3352230,19.6
1954,3352231,8.8


Erythrocyte sedimentation rate

In [149]:
ESR_view = ESR_view[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'labresulttext', 'labmeasurenamesystem']]
ESR_view = ESR_view.rename(columns={'labresultoffset': 'ESRmins', 'labresult': 'ESR', 'labresulttext' : 'ESRresulttext', 'labmeasurenamesystem' : 'ESR.unidMedida'})

filtered_df = ESR_view[(ESR_view['ESRmins'] > 0) & (ESR_view['ESRmins'] < 1441)]

ESR_view = filtered_df.groupby('patientunitstayid')['ESR'].max().reset_index()

ESR_view

,patientunitstayid,ESR
0,181906,11.0
1,243097,10.0
2,275329,3.0
3,300007,15.0
4,616272,2.0
5,654287,37.0
6,876268,42.0
7,1059195,34.0
8,1090033,18.0
9,1292649,122.0


Premature neutrophil count(bands)

In [150]:
bands_view = bands_view[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'labresulttext', 'labmeasurenamesystem']]
bands_view = bands_view.rename(columns={'labresultoffset': 'bandsmins', 'labresult': 'bands', 'labresulttext' : 'bandsresulttext', 'labmeasurenamesystem' : 'bands.unidMedida'})

filtered_df = bands_view[(bands_view['bandsmins'] > 0) & (bands_view['bandsmins'] < 1441)]

bands_view = filtered_df.groupby('patientunitstayid')['bands'].max().reset_index()

bands_view

,patientunitstayid,bands
0,151179,10.0
1,165928,9.0
2,165929,41.0
3,171925,12.0
4,171926,3.0
...,...,...
172,3211766,35.0
173,3235765,16.0
174,3237333,1.0
175,3239262,36.0


Temperature

In [151]:
Temperature_view = Temperature_view[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'labresulttext', 'labmeasurenamesystem']]
Temperature_view = Temperature_view.rename(columns={'labresultoffset': 'Tempmins', 'labresult': 'Temp', 'labresulttext' : 'Tempresulttext', 'labmeasurenamesystem' : 'Temp.unidMedida'})
Temperature_view['celsius'] = 0


filtered_df = Temperature_view[(Temperature_view['Tempmins'] > 0) & (Temperature_view['Tempmins'] < 1441)]

filtered_df = filtered_df[filtered_df['Temp'].notna()]

filtered_df['Tempresulttext'] = filtered_df['Tempresulttext'].astype(float)

filtered_df['celsius'] = filtered_df['Tempresulttext'].map(lambda x: x if x < 60 else (x - 32) * 5/9)

Temperature_view = filtered_df.groupby('patientunitstayid')['celsius'].max().reset_index()

Temperature_view

,patientunitstayid,celsius
0,250724,34.722222
1,268932,36.777778
2,272886,35.888889
3,297291,37.722222
4,302226,38.055556
...,...,...
149,2885054,37.900000
150,2894029,36.500000
151,3067309,37.000000
152,3116577,37.000000


WBC's in urine count

In [152]:
WBCurine_view = WBCurine_view[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'labresulttext', 'labmeasurenamesystem']]
WBCurine_view = WBCurine_view.rename(columns={'labresultoffset': 'WBCurinemins', 'labresult': 'WBCurine', 'labresulttext' : 'WBCurineresulttext', 'labmeasurenamesystem' : 'WBCurine.unidMedida'})

filtered_df = WBCurine_view[(WBCurine_view['WBCurinemins'] > 0) & (WBCurine_view['WBCurinemins'] < 1441)]

WBCurine_view = filtered_df.groupby('patientunitstayid')['WBCurine'].max().reset_index()

WBCurine_view

,patientunitstayid,WBCurine
0,425870,3.0
1,426976,102.0
2,443664,1.0
3,467819,30.0
4,472360,14.0
5,486824,NaN
6,490809,1.0
7,496980,1.0
8,508973,NaN
9,518155,NaN


C-reactive protein

In [153]:
CRP_view = CRP_view[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'labresulttext', 'labmeasurenamesystem']]
CRP_view = CRP_view.rename(columns={'labresultoffset': 'CRPmins', 'labresult': 'CRP', 'labresulttext' : 'CRPresulttext', 'labmeasurenamesystem' : 'CRP.unidMedida'})

filtered_df = CRP_view[(CRP_view['CRPmins'] > 0) & (CRP_view['CRPmins'] < 1441)]

CRP_view = filtered_df.groupby('patientunitstayid')['CRP'].max().reset_index()

CRP_view

,patientunitstayid,CRP
0,177689,0.40
1,238463,5.20
2,243097,708.00
3,280350,115.60
4,284517,1148.00
5,312258,3.54
6,316382,0.41
7,341782,9.58
8,345142,14.48
9,350551,20.42


lactate

In [154]:
lactate_view = lactate_view[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'labresulttext', 'labmeasurenamesystem']]
lactate_view = lactate_view.rename(columns={'labresultoffset': 'lactatemins', 'labresult': 'lactate', 'labresulttext' : 'lactateresulttext', 'labmeasurenamesystem' : 'lactate.unidMedida'})

filtered_df = lactate_view[(lactate_view['lactatemins'] > 0) & (lactate_view['lactatemins'] < 1441)]

lactate_view = filtered_df.groupby('patientunitstayid')['lactate'].max().reset_index()

lactate_view

,patientunitstayid,lactate
0,147784,1.2
1,151900,0.9
2,156308,1.5
3,162841,1.7
4,166175,2.8
...,...,...
466,3344942,3.0
467,3344944,0.8
468,3347502,1.2
469,3348292,1.5


complete inflamatory

In [155]:
inflamatory = pd.merge(WBC_view, WBCurine_view, on='patientunitstayid', how='outer')
inflamatory = pd.merge(inflamatory, CRP_view, on='patientunitstayid', how='outer')
inflamatory = pd.merge(inflamatory, lactate_view, on='patientunitstayid', how='outer')
inflamatory = pd.merge(inflamatory, Temperature_view, on='patientunitstayid', how='outer')
inflamatory = pd.merge(inflamatory, ESR_view, on='patientunitstayid', how='outer')
inflamatory = pd.merge(inflamatory, bands_view, on='patientunitstayid', how='outer')


inflamatory

,patientunitstayid,WBC,WBCurine,CRP,lactate,celsius,ESR,bands
0,141765,10.2,NaN,NaN,NaN,NaN,NaN,NaN
1,143870,11.7,NaN,NaN,NaN,NaN,NaN,NaN
2,144815,7.9,NaN,NaN,NaN,NaN,NaN,NaN
3,145427,21.1,NaN,NaN,NaN,NaN,NaN,NaN
4,147306,13.5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2009,654287,NaN,NaN,NaN,NaN,NaN,37.0,NaN
2010,1584003,NaN,NaN,NaN,NaN,NaN,13.0,NaN
2011,2581351,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,2793829,NaN,NaN,NaN,NaN,NaN,6.0,NaN


#Pulmonary

In [156]:
df_physicalExam_filtrado = df_physicalExam [['patientunitstayid', 'physicalexamoffset',  'physicalexampath', 'physicalexamvalue', 'physicalexamtext']].copy()
#Filtrar os valores adquiridos nas primeiras 24hr
df_physicalExam_24 = df_physicalExam_filtrado.loc[(df_physicalExam_filtrado["physicalexamoffset"] > 0) & (df_physicalExam_filtrado["physicalexamoffset"] < 24*60)]

#Maior e menor Oxygen saturation
ox_saturation = df_physicalExam_24[(df_physicalExam_24["physicalexamvalue"] == 'O2 Sat% Current')]
ox_saturation = ox_saturation[['patientunitstayid', 'physicalexamoffset', 'physicalexamvalue', 'physicalexamtext']]
ox_saturation.head()

,patientunitstayid,physicalexamoffset,physicalexamvalue,physicalexamtext
22,238463,19,O2 Sat% Current,97
65,162842,343,O2 Sat% Current,96
94,217838,14,O2 Sat% Current,90
132,186085,17,O2 Sat% Current,99
169,162842,195,O2 Sat% Current,95


In [157]:
df_lab_filtrado = df_lab [['patientunitstayid', 'labresultoffset', 'labname', 'labresult']].copy()
#Filtrar os valores adquiridos nas primeiras 24hr
df_lab_24 = df_lab_filtrado.loc[(df_lab_filtrado["labresultoffset"] > 0) & (df_lab_filtrado["labresultoffset"] < 24*60)]

#Partial pressure of oxygen, mean (SD), mm Hg
df_lab_pao  = df_lab_24 [(df_lab_24 ["labname"] == 'paO2')]
# Filtrar os valores mais críticos dentro das 24 horas
pao2 = df_lab_pao.groupby('patientunitstayid')['labresult'].min().reset_index()
#reorganizando a tabela
pao2 = pao2.rename(columns={'labresult': 'PAO2'})
pao2.head()

,patientunitstayid,PAO2
0,147784,76.0
1,151179,91.0
2,151900,120.0
3,162502,68.0
4,165928,64.0


In [158]:
#Respiratory rate
df_lab_fio2  = df_lab_24 [(df_lab_24 ["labname"] == 'FiO2')]
# Filtrar os valores mais críticos dentro das 24 horas
fio2 = df_lab_fio2.groupby('patientunitstayid')['labresult'].max().reset_index()
#reorganizando a tabela
fio2 = fio2.rename(columns={'labresult': 'FIO2'})
fio2.head()

,patientunitstayid,FIO2
0,147784,60.0
1,151179,28.0
2,151900,40.0
3,162502,100.0
4,169525,32.0


In [159]:
# Mesclar primeiro df_physicalExam_24 e pao2
merged_temp = pd.merge(ox_saturation, pao2, on='patientunitstayid', how='outer')
# Mesclar os datasets
pulmonary = pd.merge(merged_temp, fio2, on='patientunitstayid', how='outer')
#Substitua valores específicos por NaN
pulmonary = pulmonary.replace(-1, pd.NA)
pulmonary.head()

,patientunitstayid,physicalexamoffset,physicalexamvalue,physicalexamtext,PAO2,FIO2
0,238463,19.0,O2 Sat% Current,97,124.0,60.0
1,238463,54.0,O2 Sat% Current,95,124.0,60.0
2,162842,343.0,O2 Sat% Current,96,NaN,NaN
3,162842,195.0,O2 Sat% Current,95,NaN,NaN
4,217838,14.0,O2 Sat% Current,90,NaN,NaN


#Cardiovascular

In [160]:
# Visualizar indicadores cardiovasculares
bicarbonate = df_lab[df_lab['labname'].str.contains('bicarbonate', case=False, na=False)]
heartRate = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('HR Current', case=False, na=False)]
lactate = df_lab[df_lab['labname'].str.contains('lactate', case=False, na=False)]
systolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('systolic', case=False, na=False)]
troponin = df_lab[df_lab['labname'].str.contains('troponin', case=False, na=False)]

# Filtrar apenas resultados nas primeiras 24 horas
bicarbonate = bicarbonate.query('labresultoffset >= 0 and labresultoffset <= 1440')
heartRate = heartRate.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
lactate = lactate.query('labresultoffset >= 0 and labresultoffset <= 1440')
systolic = systolic.query('physicalexamoffset >= 0 and physicalexamoffset <= 1440')
troponin = troponin.query('labresultoffset >= 0 and labresultoffset <= 1440')

# Deixar apenas as colunas que importam para o modelo
bicarbonate = bicarbonate[['patientunitstayid', 'labresult']].copy()
heartRate = heartRate[['patientunitstayid', 'physicalexamtext']].copy()
lactate = lactate[['patientunitstayid', 'labresult']].copy()
systolic = systolic[['patientunitstayid', 'physicalexamtext']].copy()
troponin = troponin[['patientunitstayid', 'labresult']].copy()

# Remomear colunas para facilitar visualização da tabela
bicarbonate.rename(columns={'labresult': 'bicarbonate'}, inplace=True)
heartRate.rename(columns={'physicalexamtext': 'heartRate'}, inplace=True)
lactate.rename(columns={'labresult': 'lactate'}, inplace=True)
systolic.rename(columns={'physicalexamtext': 'systolic'}, inplace=True)
troponin.rename(columns={'labresult': 'troponin'}, inplace=True)

# Converter as colunas 'heartRate' e 'systolic' de string para inteiro
# Caso não dê, converte para NaN
heartRate["heartRate"] = heartRate["heartRate"].astype(int)
try:
    systolic["systolic"] = systolic["systolic"].astype(int)
except ValueError:
  systolic["systolic"] = pd.to_numeric(systolic["systolic"], errors='coerce')

# Filtrar o valor mais crítico de cada indicador
bicarbonate = bicarbonate.groupby('patientunitstayid')['bicarbonate'].min().reset_index()
heartRate = heartRate.groupby('patientunitstayid')['heartRate'].max().reset_index()
lactate = lactate.groupby('patientunitstayid')['lactate'].max().reset_index()
systolic = systolic.groupby('patientunitstayid')['systolic'].min().reset_index()
troponin = troponin.groupby('patientunitstayid')['troponin'].max().reset_index()

# Juntar as tabelas
cardiovascular = pd.merge(bicarbonate, heartRate, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, lactate, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, systolic, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, troponin, on='patientunitstayid', how='outer')

#Substitua valores específicos por NaN
cardiovascular = cardiovascular.replace(-1, pd.NA)
cardiovascular = cardiovascular.fillna(value=pd.NA)

cardiovascular

,patientunitstayid,bicarbonate,heartRate,lactate,systolic,troponin
0,141765,21.0,NaN,NaN,NaN,0.020
1,143870,22.0,NaN,NaN,96.0,NaN
2,144815,26.0,88.0,NaN,113.0,NaN
3,145427,27.0,NaN,NaN,119.0,NaN
4,147784,39.0,NaN,1.2,NaN,0.030
...,...,...,...,...,...,...
2328,1716263,NaN,NaN,NaN,NaN,0.670
2329,3067246,NaN,NaN,NaN,NaN,0.030
2330,3086799,NaN,NaN,NaN,NaN,0.010
2331,3140305,NaN,NaN,NaN,NaN,NaN


In [161]:
bicarbonate.columns

Index(['patientunitstayid', 'bicarbonate'], dtype='object')

#Renal

In [162]:
# Selecionar indicadores renais no dataset 'lab'
BUN = df_lab[df_lab['labname'].str.contains('BUN', case=False, na=False)]
BUN = df_lab[df_lab['labname'].str.contains('BUN', case=False, na=False)]
creatinine = df_lab[df_lab['labname'].str.contains('creatinine', case=False, na=False)]

# Filtrar valores adquiridos nas primeiras 24 horas de internação
BUN = BUN.query('labresultoffset >= 0 and labresultoffset <= 1440')
creatinine = creatinine.query('labresultoffset >=0 and labresultoffset <= 1440')

# Criar uma cópia com as colunas importantes para o modelo
BUN = BUN[['patientunitstayid', 'labresult']].copy()
creatinine = creatinine[['patientunitstayid','labresult']].copy()

# Renomear as colunas para facilitar visualização
BUN.rename(columns={'labresult' : 'bloodUreaNitrogen'}, inplace=True)
creatinine.rename(columns={'labresult' : 'creatinine'}, inplace=True)

# Filtrar os valores mais críticos dentro das 24 horas
BUN = BUN.groupby('patientunitstayid')['bloodUreaNitrogen'].max().reset_index()
creatinine = creatinine.groupby('patientunitstayid')['creatinine'].max().reset_index()

# Mesclar os datasets
renal = pd.merge(BUN, creatinine, on='patientunitstayid', how='outer')

#Substitua valores específicos por NaN
renal = renal.replace(-1, pd.NA)
renal = renal.fillna(value=pd.NA)

renal

,patientunitstayid,bloodUreaNitrogen,creatinine
0,141765,28.0,1.04
1,143870,14.0,1.14
2,144815,6.0,0.63
3,145427,14.0,1.05
4,147784,17.0,0.64
...,...,...,...
2095,1680400,NaN,0.82
2096,2192084,NaN,1.00
2097,2607501,NaN,0.19
2098,2953061,NaN,4.07


#Hepatic

In [163]:
# Selecionar indicadores hepáticos no dataset 'lab'
alanine = df_lab[df_lab['labname'].str.contains('ALT', case=False, na=False)]
aspartate = df_lab[df_lab['labname'].str.contains('AST', case=False, na=False)]
bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]

# Filtrar para o intervalo das primeiras 24 horas do paciente na UTI
alanine = alanine.query('labresultoffset >= 0 and labresultoffset <= 1440')
aspartate = aspartate.query('labresultoffset >= 0 and labresultoffset <= 1440')
bilirubin = bilirubin.query('labresultoffset >= 0 and labresultoffset <= 1440')

# Fazer uma cópia com as colunas importantes para o modelo
alanine = alanine[['patientunitstayid', 'labresult']].copy()
aspartate = aspartate[['patientunitstayid', 'labresult']].copy()
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()

# Renomear colunas para facilitar visualização
alanine.rename(columns={'labresult' : 'alanine'}, inplace=True)
aspartate.rename(columns={'labresult' : 'aspartate'}, inplace=True)
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)

# Filtrar os valores mais críticos de cada paciente
alanine = alanine.groupby('patientunitstayid')['alanine'].max().reset_index()
aspartate = aspartate.groupby('patientunitstayid')['aspartate'].max().reset_index()
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

# Mesclar os datasets
hepatic = pd.merge(alanine, aspartate, on='patientunitstayid', how='outer')
hepatic = pd.merge(hepatic, bilirubin, on='patientunitstayid', how='outer')

# Substituir valores específicos por NaN
hepatic = hepatic.replace(-1, pd.NA)
hepatic = hepatic.fillna(value=pd.NA)

hepatic

,patientunitstayid,alanine,aspartate,bilirubin
0,144815,42.0,39.0,0.5
1,147784,35.0,20.0,0.3
2,149713,18.0,17.0,0.5
3,151179,16.0,60.0,0.2
4,152954,18.0,42.0,0.4
...,...,...,...,...
981,409048,NaN,NaN,0.3
982,414364,NaN,NaN,0.8
983,418417,NaN,NaN,0.6
984,419678,NaN,NaN,0.5


#Hematologic

In [164]:
# Selecionar indicadores hematológicos no dataset 'lab'
hemoglobin = df_lab[df_lab['labname'].str.contains('Hgb', case=False, na=False)]
iqr = df_lab[df_lab['labname'].str.contains('INR', case=False, na=False)]
platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]

# Filtrar os resultados das primeiras 24 horas
hemoglobin = hemoglobin.query('labresultoffset >= 0 and labresultoffset <= 1440')
iqr = iqr.query('labresultoffset >= 0 and labresultoffset <= 1440')
platelets = platelets.query('labresultoffset >= 0 and labresultoffset <= 1440')

# Criar cópias dos datasets com as colunas importantes para o modelo
hemoglobin = hemoglobin[['patientunitstayid', 'labresult']].copy()
iqr = iqr[['patientunitstayid', 'labresult']].copy()
platelets = platelets[['patientunitstayid', 'labresult']].copy()

# Renomear colunas para facilitar visualização
hemoglobin.rename(columns={'labresult' : 'hemoglobin'}, inplace=True)
iqr.rename(columns={'labresult' : 'iqr'}, inplace=True)
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)

# Filtrar os valores mais críticos entre as 24 horas
hemoglobin = hemoglobin.groupby('patientunitstayid')['hemoglobin'].min().reset_index()
iqr = iqr.groupby('patientunitstayid')['iqr'].max().reset_index()
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

# Mesclar os datasets
hematologic = pd.merge(hemoglobin, iqr, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, platelets, on='patientunitstayid', how='outer')

# Substituir valores específicos por NaN
hematologic = hematologic.replace(-1, pd.NA)
hematologic = hematologic.fillna(value=pd.NA)

hematologic

,patientunitstayid,hemoglobin,iqr,platelets
0,141765,12.3,NaN,191.0
1,143870,12.3,NaN,152.0
2,144815,12.3,1.00,319.0
3,145427,13.3,NaN,196.0
4,147306,12.1,NaN,280.0
...,...,...,...,...
2007,3021108,NaN,0.91,NaN
2008,3027283,NaN,1.16,NaN
2009,3067246,NaN,1.00,NaN
2010,755285,NaN,NaN,136.0


#Other

In [165]:
#Albumin
df_lab_albumin  = df_lab_24 [(df_lab_24 ["labname"] == 'albumin')]
# Filtrar os valores mais críticos dentro das 24 horas
albumin = df_lab_albumin.groupby('patientunitstayid')['labresult'].max().reset_index()
#reorganizando a tabela
albumin = albumin.rename(columns={'labresult': 'Albumin'})
albumin.head()

,patientunitstayid,Albumin
0,144815,3.6
1,147784,2.9
2,149713,3.8
3,151179,1.3
4,152954,2.4


In [166]:
#Chloride
df_lab_chloride  = df_lab_24 [(df_lab_24 ["labname"] == 'chloride')]
# Filtrar os valores mais críticos dentro das 24 horas
chloride = df_lab_chloride.groupby('patientunitstayid')['labresult'].max().reset_index()
#reorganizando a tabela
chloride = chloride.rename(columns={'labresult': 'Chloride'})
chloride.head()

,patientunitstayid,Chloride
0,141765,105.0
1,143870,106.0
2,144815,105.0
3,145427,109.0
4,147784,102.0


In [167]:
#Glucose
df_lab_glucose  = df_lab_24 [(df_lab_24 ["labname"] == 'glucose')]
# Filtrar os valores mais críticos dentro das 24 horas
glucose = df_lab_glucose.groupby('patientunitstayid')['labresult'].max().reset_index()
#reorganizando a tabela
glucose = glucose.rename(columns={'labresult': 'Clucose'})
glucose.head()

,patientunitstayid,Clucose
0,141765,61.0
1,143870,140.0
2,144815,82.0
3,145427,139.0
4,147784,144.0


In [168]:
#Sodium,
df_lab_sodium  = df_lab_24 [(df_lab_24 ["labname"] == 'sodium')]
# Filtrar os valores mais críticos dentro das 24 horas
sodium = df_lab_sodium.groupby('patientunitstayid')['labresult'].max().reset_index()
#reorganizando a tabela
sodium = sodium.rename(columns={'labresult': 'Sodium'})
sodium.head()

,patientunitstayid,Sodium
0,141765,139.0
1,143870,139.0
2,144815,141.0
3,145427,141.0
4,147784,143.0


In [169]:
#GCS

motor = df_physicalExam_24[df_physicalExam_24['physicalexampath'].str.contains('Motor Score', case=False, na=False)]
#excluir colunas desnecessárias
motor = motor.drop(columns='physicalexamvalue')
motor = motor.drop(columns='physicalexampath')
#organizar tabela
motor = motor.rename(columns={'physicalexamtext': 'motor'})
motor['motor'] = motor['motor'].astype(int)


eyes = df_physicalExam_24[df_physicalExam_24['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
#excluir colunas desnecessárias
eyes = eyes.drop(columns='physicalexamoffset')
eyes = eyes.drop(columns='physicalexampath')
eyes = eyes.drop(columns='physicalexamvalue')
#organizar tabela
eyes = eyes.rename(columns={'physicalexamtext': 'eyes'})
eyes['eyes'] = eyes['eyes'].astype(int)

verbal = df_physicalExam_24[df_physicalExam_24['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
#excluir colunas desnecessárias
verbal = verbal.drop(columns='physicalexamoffset')
verbal = verbal.drop(columns='physicalexampath')
verbal = verbal.drop(columns='physicalexamvalue')
#organizar tabela
verbal = verbal.rename(columns={'physicalexamtext': 'verbal'})
verbal['verbal'] = verbal['verbal'].astype(int)

# Mesclar primeiro df_physicalExam_24 e pao2
gcs_temp = pd.merge(motor, eyes, on='patientunitstayid', how='outer')
# Mesclar os datasets
gcs = pd.merge(gcs_temp, verbal, on='patientunitstayid', how='outer')
gcs ['gcs total'] = gcs ['motor'] + gcs ['eyes'] + gcs ['verbal']

# Filtrar os valores mais críticos dentro das 24 horas
idx = gcs.groupby('patientunitstayid')['gcs total'].idxmax()
gcs = gcs.loc[idx]

gcs.head()

,patientunitstayid,physicalexamoffset,motor,eyes,verbal,gcs total
57,141765,7,6,4,4,14
54,143870,4,6,4,4,14
98,144815,32,6,4,4,14
102,145427,8,6,4,4,14
25,147307,53,6,4,4,14


In [170]:
# Mesclar primeiro df_physicalExam_24 e pao2
other_temp1 = pd.merge(albumin, chloride, on='patientunitstayid', how='outer')
# Mesclar os datasets
other_temp2 = pd.merge(other_temp1, glucose, on='patientunitstayid', how='outer')
# Mesclar os datasets
other_temp3 = pd.merge(other_temp2, sodium, on='patientunitstayid', how='outer')
# Mesclar os datasets
other = pd.merge(other_temp3, gcs, on='patientunitstayid', how='outer')
#Substitua valores específicos por NaN
other = other.replace(-1, pd.NA)
other.head()

,patientunitstayid,Albumin,Chloride,Clucose,Sodium,physicalexamoffset,motor,eyes,verbal,gcs total
0,144815,3.6,105.0,82.0,141.0,32.0,6.0,4.0,4.0,14.0
1,147784,2.9,102.0,144.0,143.0,10.0,6.0,4.0,4.0,14.0
2,149713,3.8,99.0,97.0,133.0,4.0,6.0,4.0,4.0,14.0
3,151179,1.3,115.0,99.0,148.0,18.0,6.0,4.0,4.0,14.0
4,152954,2.4,105.0,146.0,137.0,3.0,6.0,4.0,4.0,14.0


#Outcomes

In [171]:
#Mortalidade
mortalidade = df_patient[['patientunitstayid', 'apacheadmissiondx', 'unitdischargestatus']].copy()
mortalidade.head()

,patientunitstayid,apacheadmissiondx,unitdischargestatus
0,141764,NaN,Alive
1,141765,"Rhythm disturbance (atrial, supraventricular)",Alive
2,143870,"Endarterectomy, carotid",Alive
3,144815,"Overdose, other toxin, poison or drug",Alive
4,145427,"GI perforation/rupture, surgery for",Alive


#Merge Set

In [172]:
#Internações: 0 por outro diagnóstico e 1 para Sepse
diagnostico = df_patient [['patientunitstayid', 'apacheadmissiondx']].copy()

#Criar a coluna de diagnóstico de sepse ou não
diagnostico['sepse_diagnostico'] = 0

# Verificar se ocorre a palavra "sepse" na coluna 'apacheadmissiondx' e atribuir 1 se ocorrer
diagnostico.loc[diagnostico['apacheadmissiondx'].str.contains('sepsis', case=False, na=False), 'sepse_diagnostico'] = 1

diagnostico.head()

,patientunitstayid,apacheadmissiondx,sepse_diagnostico
0,141764,NaN,0
1,141765,"Rhythm disturbance (atrial, supraventricular)",0
2,143870,"Endarterectomy, carotid",0
3,144815,"Overdose, other toxin, poison or drug",0
4,145427,"GI perforation/rupture, surgery for",0


In [173]:
df_final = pd.merge(characteristic, sofa, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, inflamatory, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, pulmonary, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, cardiovascular, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, renal, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, hepatic, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, hematologic, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, other, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, diagnostico, on='patientunitstayid', how='outer')
df_final.columns


Index(['patientunitstayid', 'gender', 'age', 'ethnicity', 'sofascore', 'WBC',
       'WBCurine', 'CRP', 'lactate_x', 'celsius', 'ESR', 'bands',
       'physicalexamoffset_x', 'physicalexamvalue', 'physicalexamtext', 'PAO2',
       'FIO2', 'bicarbonate', 'heartRate', 'lactate_y', 'systolic', 'troponin',
       'bloodUreaNitrogen', 'creatinine', 'alanine', 'aspartate', 'bilirubin',
       'hemoglobin', 'iqr', 'platelets', 'Albumin', 'Chloride', 'Clucose',
       'Sodium', 'physicalexamoffset_y', 'motor', 'eyes', 'verbal',
       'gcs total', 'apacheadmissiondx', 'sepse_diagnostico'],
      dtype='object')

In [174]:
df_final[df_final['sepse_diagnostico']==1]

,patientunitstayid,gender,age,ethnicity,sofascore,WBC,WBCurine,CRP,lactate_x,celsius,...,Chloride,Clucose,Sodium,physicalexamoffset_y,motor,eyes,verbal,gcs total,apacheadmissiondx,sepse_diagnostico
13,151179,Female,59,Caucasian,1.0,49.8,NaN,NaN,NaN,NaN,...,115.0,99.0,148.0,18.0,6.0,4.0,4.0,14.0,"Sepsis, cutaneous/soft tissue",1
15,151900,Female,66,Caucasian,3.0,7.3,NaN,NaN,0.9,NaN,...,109.0,139.0,140.0,NaN,NaN,NaN,NaN,NaN,"Sepsis, pulmonary",1
19,156308,Male,87,Caucasian,7.0,NaN,NaN,NaN,1.5,NaN,...,107.0,446.0,144.0,39.0,6.0,4.0,4.0,14.0,"Sepsis, pulmonary",1
33,162841,Female,84,Caucasian,8.0,19.7,NaN,NaN,1.7,NaN,...,113.0,95.0,146.0,2.0,6.0,4.0,4.0,14.0,"Sepsis, pulmonary",1
47,166175,Male,60,Caucasian,11.0,12.9,NaN,NaN,2.8,NaN,...,99.0,285.0,132.0,37.0,6.0,4.0,4.0,14.0,"Sepsis, cutaneous/soft tissue",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2736,3327376,Male,56,Caucasian,NaN,18.2,NaN,NaN,3.4,NaN,...,106.0,226.0,134.0,49.0,6.0,4.0,4.0,14.0,"Sepsis, unknown",1
2737,3329305,Male,72,Caucasian,NaN,37.0,NaN,17.34,NaN,NaN,...,108.0,118.0,138.0,19.0,6.0,4.0,4.0,14.0,"Sepsis, renal/UTI (including bladder)",1
2750,3341168,Male,61,African American,NaN,18.9,NaN,NaN,NaN,NaN,...,99.0,97.0,136.0,70.0,6.0,4.0,4.0,14.0,"Sepsis, pulmonary",1
2751,3341168,Male,61,African American,NaN,18.9,NaN,NaN,NaN,NaN,...,99.0,97.0,136.0,70.0,6.0,4.0,4.0,14.0,"Sepsis, pulmonary",1
